In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer

from datetime import datetime, timedelta

import lightgbm as lgb
import wandb

import warnings # Supress warnings
warnings.filterwarnings("ignore")

In [5]:
train_df=pd.read_csv('New folder\imp_train.csv')
test_df=pd.read_csv('New folder\imp_test.csv')
display(train_df.info())
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375734 entries, 0 to 375733
Columns: 263 entries, Unnamed: 0 to quarter_cos
dtypes: float64(253), int64(9), object(1)
memory usage: 753.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31354 entries, 0 to 31353
Columns: 263 entries, Unnamed: 0 to quarter_cos
dtypes: float64(253), int64(9), object(1)
memory usage: 62.9+ MB


In [7]:
non_numeric=train_df.select_dtypes(include=['object']).columns.tolist()
non_numeric

['startdate']

In [89]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [90]:
model = LinearRegression()
train_df1=train_df.drop('startdate',axis=1)
test_df1=test_df.drop(['startdate','contest-tmp2m-14d__tmp2m'],axis=1)
test_df1.shape

(31354, 261)

In [91]:
X=train_df1.drop('contest-tmp2m-14d__tmp2m',axis=1)
y=train_df['contest-tmp2m-14d__tmp2m']

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [93]:
model.fit(X_train, y_train)

LinearRegression()

In [94]:
y_pred=model.predict(X_test)

In [95]:
from sklearn.metrics import mean_squared_error

# Evaluate the model's accuracy
rmse= mean_squared_error(y_test, y_pred)
print("rmse:", rmse)

rmse: 1.7265408043846568



### xg boost

In [96]:
import xgboost as xgb

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=7)

In [154]:
def reg_function(n,r,g,md):
    regressor = xgb.XGBRegressor(
        n_estimators=n,#100
        reg_lambda=r,#1
        gamma=g,#0
        max_depth=md#4
    )

    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)

    rmse= mean_squared_error(y_test, y_pred)

    return rmse
    

    

In [155]:
X_test.shape

(150294, 261)

In [156]:
rr={}
n,r,g,i=100,1,0,4
for i in range(7,13):
    rr[i]=reg_function(n,r,g,i)
print(rr)

{7: 0.4118251693181089, 8: 0.33037900402147113, 9: 0.273815473652608, 10: 0.2321910781253381, 11: 0.20638769775783997, 12: 0.19025538221794291}


In [157]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=12
)

In [158]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [163]:
y_pred = regressor.predict(X_test)
y_pred

array([-1.8485708, 12.901185 , -7.072723 , ..., 27.670103 , 13.422303 ,
       17.686182 ], dtype=float32)

In [164]:
rmse= mean_squared_error(y_test, y_pred)
print("rmse:", rmse)

rmse: 0.19025538221794291


In [165]:
sub_pred_y=regressor.predict(test_df1)
sub_pred_y

array([28.972061 , 28.941647 , 29.762823 , ...,  2.4971945,  2.4395573,
        2.9072802], dtype=float32)

In [166]:
submission = pd.read_csv('sample_solution.csv')
submission['contest-tmp2m-14d__tmp2m'] = sub_pred_y
submission.to_csv('submission12.csv', 
                  index = False) # Set index to false to avoid issues in evaluation
display(submission)

,contest-tmp2m-14d__tmp2m,index
0,28.972061,375734
1,28.941647,375735
2,29.762823,375736
3,30.081388,375737
4,29.235151,375738
...,...,...
31349,1.284986,407083
31350,2.737961,407084
31351,2.497195,407085
31352,2.439557,407086
